# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 6 2023 11:59AM,254465,15,8994553,"Mizner Bioscience, LLC",Released
1,Jan 6 2023 11:59AM,254465,15,8994555,"Mizner Bioscience, LLC",Released
2,Jan 6 2023 11:59AM,254465,15,8994556,"Mizner Bioscience, LLC",Released
3,Jan 6 2023 11:57AM,254464,10,0086284786,ISDIN Corporation,Released
4,Jan 6 2023 11:57AM,254464,10,0086284793,ISDIN Corporation,Released
5,Jan 6 2023 11:57AM,254464,10,0086284797,ISDIN Corporation,Released
6,Jan 6 2023 11:57AM,254464,10,0086284798,ISDIN Corporation,Released
7,Jan 6 2023 11:57AM,254464,10,0086284806,ISDIN Corporation,Released
8,Jan 6 2023 11:55AM,254463,16,8994514,Sartorius Bioprocess Solutions,Released
9,Jan 6 2023 11:55AM,254463,16,8994526,Sartorius Bioprocess Solutions,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,254461,Released,1
46,254462,Released,2
47,254463,Released,4
48,254464,Released,5
49,254465,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254461,NaN,NaN,1.0
254462,NaN,NaN,2.0
254463,NaN,NaN,4.0
254464,NaN,NaN,5.0
254465,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254351,0.0,0.0,2.0
254357,0.0,0.0,1.0
254361,12.0,10.0,6.0
254367,0.0,8.0,22.0
254373,0.0,2.0,33.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254351,0,0,2
254357,0,0,1
254361,12,10,6
254367,0,8,22
254373,0,2,33


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254351,0,0,2
1,254357,0,0,1
2,254361,12,10,6
3,254367,0,8,22
4,254373,0,2,33


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254351,,,2
1,254357,,,1
2,254361,12,10,6
3,254367,,8,22
4,254373,,2,33


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 6 2023 11:59AM,254465,15,"Mizner Bioscience, LLC"
3,Jan 6 2023 11:57AM,254464,10,ISDIN Corporation
8,Jan 6 2023 11:55AM,254463,16,Sartorius Bioprocess Solutions
12,Jan 6 2023 11:44AM,254462,15,"Brookfield Pharmaceuticals, LLC"
14,Jan 6 2023 11:44AM,254461,15,"Alliance Pharma, Inc."
15,Jan 6 2023 11:37AM,254460,19,"NAPP Technologies, LLC"
16,Jan 6 2023 11:05AM,254458,18,Innova Softgel LLC
17,Jan 6 2023 10:58AM,254457,20,"HVL, LLC dba Atrium Innovations"
18,Jan 6 2023 10:39AM,254455,19,"NAPP Technologies, LLC"
19,Jan 6 2023 10:30AM,254453,20,"HVL, LLC dba Atrium Innovations"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 6 2023 11:59AM,254465,15,"Mizner Bioscience, LLC",,,3
1,Jan 6 2023 11:57AM,254464,10,ISDIN Corporation,,,5
2,Jan 6 2023 11:55AM,254463,16,Sartorius Bioprocess Solutions,,,4
3,Jan 6 2023 11:44AM,254462,15,"Brookfield Pharmaceuticals, LLC",,,2
4,Jan 6 2023 11:44AM,254461,15,"Alliance Pharma, Inc.",,,1
5,Jan 6 2023 11:37AM,254460,19,"NAPP Technologies, LLC",,,1
6,Jan 6 2023 11:05AM,254458,18,Innova Softgel LLC,,,1
7,Jan 6 2023 10:58AM,254457,20,"HVL, LLC dba Atrium Innovations",,,1
8,Jan 6 2023 10:39AM,254455,19,"NAPP Technologies, LLC",,,1
9,Jan 6 2023 10:30AM,254453,20,"HVL, LLC dba Atrium Innovations",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 11:59AM,254465,15,"Mizner Bioscience, LLC",3,,
1,Jan 6 2023 11:57AM,254464,10,ISDIN Corporation,5,,
2,Jan 6 2023 11:55AM,254463,16,Sartorius Bioprocess Solutions,4,,
3,Jan 6 2023 11:44AM,254462,15,"Brookfield Pharmaceuticals, LLC",2,,
4,Jan 6 2023 11:44AM,254461,15,"Alliance Pharma, Inc.",1,,
5,Jan 6 2023 11:37AM,254460,19,"NAPP Technologies, LLC",1,,
6,Jan 6 2023 11:05AM,254458,18,Innova Softgel LLC,1,,
7,Jan 6 2023 10:58AM,254457,20,"HVL, LLC dba Atrium Innovations",1,,
8,Jan 6 2023 10:39AM,254455,19,"NAPP Technologies, LLC",1,,
9,Jan 6 2023 10:30AM,254453,20,"HVL, LLC dba Atrium Innovations",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 11:59AM,254465,15,"Mizner Bioscience, LLC",3,,
1,Jan 6 2023 11:57AM,254464,10,ISDIN Corporation,5,,
2,Jan 6 2023 11:55AM,254463,16,Sartorius Bioprocess Solutions,4,,
3,Jan 6 2023 11:44AM,254462,15,"Brookfield Pharmaceuticals, LLC",2,,
4,Jan 6 2023 11:44AM,254461,15,"Alliance Pharma, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 11:59AM,254465,15,"Mizner Bioscience, LLC",3.0,NaN,NaN
1,Jan 6 2023 11:57AM,254464,10,ISDIN Corporation,5.0,NaN,NaN
2,Jan 6 2023 11:55AM,254463,16,Sartorius Bioprocess Solutions,4.0,NaN,NaN
3,Jan 6 2023 11:44AM,254462,15,"Brookfield Pharmaceuticals, LLC",2.0,NaN,NaN
4,Jan 6 2023 11:44AM,254461,15,"Alliance Pharma, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 11:59AM,254465,15,"Mizner Bioscience, LLC",3.0,0.0,0.0
1,Jan 6 2023 11:57AM,254464,10,ISDIN Corporation,5.0,0.0,0.0
2,Jan 6 2023 11:55AM,254463,16,Sartorius Bioprocess Solutions,4.0,0.0,0.0
3,Jan 6 2023 11:44AM,254462,15,"Brookfield Pharmaceuticals, LLC",2.0,0.0,0.0
4,Jan 6 2023 11:44AM,254461,15,"Alliance Pharma, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4579336,287.0,23.0,13.0
15,1272145,11.0,0.0,0.0
16,1017717,7.0,0.0,0.0
18,508878,2.0,0.0,0.0
19,2035444,20.0,2.0,0.0
20,1272092,4.0,1.0,0.0
21,254398,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4579336,287.0,23.0,13.0
1,15,1272145,11.0,0.0,0.0
2,16,1017717,7.0,0.0,0.0
3,18,508878,2.0,0.0,0.0
4,19,2035444,20.0,2.0,0.0
5,20,1272092,4.0,1.0,0.0
6,21,254398,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,287.0,23.0,13.0
1,15,11.0,0.0,0.0
2,16,7.0,0.0,0.0
3,18,2.0,0.0,0.0
4,19,20.0,2.0,0.0
5,20,4.0,1.0,0.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,287.0
1,15,Released,11.0
2,16,Released,7.0
3,18,Released,2.0
4,19,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,13.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,23.0,0.0,0.0,0.0,2.0,1.0,0.0
Released,287.0,11.0,7.0,2.0,20.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,13.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,23.0,0.0,0.0,0.0,2.0,1.0,0.0
2,Released,287.0,11.0,7.0,2.0,20.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,13.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,23.0,0.0,0.0,0.0,2.0,1.0,0.0
2,Released,287.0,11.0,7.0,2.0,20.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()